In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")


from pyspark.sql.functions import *
from pyspark.sql.types import *



import folium
import html
import pickle
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.session import SparkSession

In [0]:
!java -version
!sudo update-alternatives --config java
!java -version

openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (b

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
with open('gdrive/My Drive/Yelp Data/dfRestaurants.pickle', 'rb') as f:
    df_business = pickle.load(f)

In [0]:
sc = SparkContext('local')
spark = SparkSession(sc)

In [0]:
df_business_spark = spark.createDataFrame(df_business)

In [0]:
users_df = pd.read_csv('gdrive/My Drive/Yelp Data/yelp_user.csv')

In [0]:
users_df = users_df.drop(columns = ['review_count','average_stars','name','yelping_since','useful','funny','cool','fans','elite','compliment_hot','compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note','compliment_plain','compliment_funny','compliment_cool','compliment_writer','compliment_photos',])

In [0]:
users_df = users_df.mask(users_df.eq('None')).dropna()


In [0]:
users_df.sample(50000)

,user_id,friends
31348,8gYdZVFSA94HopPEim4lFg,"tl8eYxepwIGpmJeZgDz2SQ, q0nrt3fePvXJQFM4ykGYqg"
435251,M59WtfQKPkDIHqTZ3HNq7w,"UiE3CZNJvJsjUgjJtShDcQ, C8UaHvbFLpi6u887QI_bFg..."
679642,Bz192TbD61n5pVc76b8pLA,"oRU6APpVlLFlAGpwrJKs0g, 03iMNggjXP2tOkfih15ccA..."
1066469,Jd4auX83JlwWsvxlOp17jQ,"RUO46LMpc9Nodz3lQViVTw, GwbbCmGsLmyp7mjtNVWg6Q..."
328416,o3yE9gLnRVog48fKnUTyCQ,"HkfoTQhGp5Jepw8hjqODGw, IrdHOSUP_Lyvp1-h4ZOByw..."
...,...,...
1786,TA8lMiUg95s2cveylmdVew,"iaoVfssac6w3vE4Mt7ZYbA, ZybmFQk5r7er6AkTJGkAlQ..."
409935,SD35oARo4MJoYO4_AVRXCg,"TiZ1j7NuJgqtgUPB29NbPg, kHAgdAPmzY7rNekrFFNLBQ..."
488292,Wy_X5_1YWSx1kcr3qv9Hrw,"n6TKnAIGZPHgWLBbA-UV5g, F2Xxkh_AvLnnMi9W_rgprw..."
1138359,Qg_usUtNQzSxV8-noF6Sew,"jVLK7q2AYMmrwb2KA5aetA, L-g_wc7WJdJ-dMFiQrPp8A..."


In [0]:
df_users_spark = spark.createDataFrame(users_df)

In [0]:
u_id = 'uxKSnOVAoEj4I6X9YhLBlg'

query = """
select business_id, count(*) as 4_5_stars_count 
from reviews
where user_id in
    (select f.friend_id from friends f
    inner join users u on f.friend_id = u.user_id
    where f.user_id = "{}") 
and stars >= 4 
and business_id not in (select business_id from reviews where user_id = "{}")
group by business_id
order by count(*) desc limit 100
""".format(u_id, u_id)

friend_recoms_df = sqlContext.sql(query)

friend_recoms_df.toPandas()

,business_id,4_5_stars_count
15,l09JfMeQ6ynYs5MCJtrcmQ,4
25,1K4qrnfyzKzGgJPBEcJaNQ,5
96,dTWfATVrBfKj7Vdn0qWVWg,4
104,1nhf9BPXOBFBkbRkpsFaxA,4
113,sJ0MYSAIVK28cMzh-s-NPA,5
...,...,...
174507,3zIy2T1bRwxvK-dRmopS_g,4
174518,sEAKw3MZkER1u_1fzIeD3g,5
174519,1HplwLVbBid-Bgw1sEPGFg,4
174536,xqNpOnxY0XzdnUE4sjmg0w,4
